In [39]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import datetime as dt
import statsmodels.api as sm
from IPython import get_ipython
from statsmodels.formula.api import ols
import random
import time
from datetime import date, timedelta
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from scipy.stats import poisson
import scipy.stats as stats
%matplotlib inline

from google.oauth2.service_account import Credentials
import google_auth_httplib2
import pygsheets
import df2gspread as d2g
import googleapiclient
from googleapiclient  import discovery
from httplib2 import Http
from oauth2client import file, client, tools
from gspread_formatting.dataframe import format_with_dataframe, BasicFormatter
from gspread_formatting import Color
import logging

from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
import gslides
from gslides import (Frame,Presentation,Spreadsheet,Table,Series,Chart)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

import sqlalchemy as db
from sqlalchemy import create_engine
import mysql.connector
import psycopg2
import gspread

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
path ='C:/Users/mncedisi.mncwabe/Documents/Connect to BQ/tal-production-data-4390ad039b75.json'
credentials = Credentials.from_service_account_file(path)


#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

# import orders list from google sheet 
scope_app = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive'] 
cred = ServiceAccountCredentials.from_json_keyfile_name(path,scope_app) 
# authorize the clientsheet 
client = gspread.authorize(cred)
sheet = client.open_by_key('103l-5z1Ono33ftZydlGLYJ07KTU7LQnjCmuAH0bciDY')
sheet_instance = sheet.get_worksheet(0)
records = sheet_instance.get_all_records()
orders = pd.DataFrame (records, columns = ['idOrder'])

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

orders_list = list(orders.idOrder.unique())
#convert the list to a tuple
t = tuple(orders_list)
chunkv = 50000
df_orders = []
for i in tqdm(range(0, len(t), chunkv)):
    query = f'''SELECT 
    FORMAT_DATE("%Y-%m-%d",order_authed_timestamp) AS Auth_Date,
    o.order_id,
    oi.order_item_id,
    oi.is_deal,
    oi.deal_id,
    oi.product_id,
    bh.tsin_id,
    bh.bh_division,
    bh.bh_reporting_department,
    bh.bh_level_1,
    bh.bh_level_2,
    bh.is_marketplace,
    oi.order_item_quantity as QTY_Sold,
    oi.sold_price as Item_UnitPrice,
    oi.order_item_sales_inc as Total_inc_vat,
    oi.order_item_sales as Total_excl_vat,
    oi.sold_wacp,
    (oi.sold_wacp)*(oi.order_item_quantity) as OrderCostPrice,
    item_unit_subsidies,
    (oi.item_unit_subsidies) *(oi.order_item_quantity) as TotalSubsPerProduct,
    oi.order_item_subsidies,
    oi.order_item_pure_product_margin as ppm,
    bh.tsin_title

FROM `tal-production-data-bi.tal_dm_order.orders_2020_2030` o,
    UNNEST(o.order_items) oi
LEFT JOIN `tal-production-data-bi.tal_dm_product.business_hierarchy` bh
    ON bh.product_id = oi.product_id
WHERE   
    DATE(o.order_placed_timestamp) >= "2022-05-01"
    AND DATE(o.order_authed_timestamp) >= "2022-05-01" 
    AND o.order_authorisation_status IN ('Auth')
    AND o.is_not_return_cancel_in_full IS TRUE
    AND is_test_customer = false
    AND oi.order_item_status LIKE 'Shipped'
    AND o.order_id IN {t[i:i + chunkv]}
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22, 23
       '''
    dfv = pd.read_gbq(query, project_id='gcp-takealot', dialect='standard', credentials=credentials)
    df_orders.append(dfv)

df_orders = pd.concat(df_orders)

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


# Export to google sheets
start_time = time.time()

file = 'C:/Users/mncedisi.mncwabe/Documents/Connect to BQ/tal-production-data-4390ad039b75.json'
sheet_id = '17hJebWbLtOTaX7TncuKc_fruGz46WCXMXpc02yi3n-8'
sheet_name = 'ppm'

def write_to_gsheet(service_file_path, spreadsheet_id, sheet_name, data_df):
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    try:
        sh.add_worksheet(sheet_name)
    except:
        pass
    wks_write = sh.worksheet_by_title(sheet_name)
    wks_write.clear('A1',None,'*')
    wks_write.set_dataframe(data_df, (1,1), encoding='utf-8', fit=True)
    wks_write.frozen_rows = 1

write_to_gsheet(file, sheet_id, sheet_name, df_orders)

print("--- %s seconds ---" % (time.time() - start_time))

100%|██████████| 1/1 [00:12<00:00, 12.50s/it]


--- 3.4525110721588135 seconds ---
